In [13]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

INPUT_FILE = "Uitgebreide_VKM_dataset.csv"
OUTPUT_FILE = "Uitgebreide_VKM_dataset_cleaned.csv"

# Kolommen die we als tekst willen schoonmaken
TEXT_COLS = [
    "shortdescription",
    "description",
    "content",
    "learningoutcomes"
]

# Stopwoorden en lemmatizer
# Kies 'english' of 'dutch' afhankelijk van je dataset
STOP_LANG = "dutch"   # of "english"
stop_words = set(stopwords.words(STOP_LANG))
lemmatizer = WordNetLemmatizer()

def normalize_length(tokens, max_len=200):
    """Beperk tokens tot een vaste maximale lengte om scheefheid te verminderen."""
    return tokens[:max_len]


def normalize_text(text: str) -> str:
    """Maak tekst schoon, lemmatiseer en normaliseer lengte."""
    if pd.isna(text):
        return ""

    text = str(text).lower()

    # Verwijder alles behalve letters, cijfers en spaties
    text = re.sub(r"[^a-zA-Z0-9\sáéíóúàèìòùäëïöüâêîôûçñ]", " ", text)

    tokens = word_tokenize(text)

    # Filter op stopwoorden en korte tokens
    tokens = [t for t in tokens if t not in stop_words and len(t) > 2]

    # Lemmatiseer
    tokens = [lemmatizer.lemmatize(t) for t in tokens]

    tokens = normalize_length(tokens, max_len=200)

    return " ".join(tokens)



def clean_vkm_dataset(input_file: str, output_file: str) -> pd.DataFrame:
    # Data inladen
    df = pd.read_csv(input_file)

    print("=" * 60)
    print("DATA CLEANING PROCESS")
    print("=" * 60)
    print(f"Origineel: {df.shape[0]} rijen, {df.shape[1]} kolommen")

    df_cleaned = df.copy()

    # 1. Verwijder de kleur-kolommen (Rood, Groen, Blauw, Geel) als ze bestaan
    kleur_kolommen = ["Rood", "Groen", "Blauw", "Geel"]
    bestaande_kleuren = [c for c in kleur_kolommen if c in df_cleaned.columns]
    if bestaande_kleuren:
        df_cleaned = df_cleaned.drop(columns=bestaande_kleuren)
        print(f"\n1. Kleur-kolommen verwijderd: {bestaande_kleuren}")
    else:
        print("\n1. Geen kleur-kolommen gevonden om te verwijderen")

    print(f"   -> {df_cleaned.shape[1]} kolommen resterend")

    # 2. Vervang "Ntb" waarden in alle tekstvelden
    print("\n2. Vervang 'Ntb' waarden in tekstvelden")
    ntb_replacements = {}
    
    # Voor shortdescription: gebruik description (eerste 200 chars)
    if "shortdescription" in df_cleaned.columns and "description" in df_cleaned.columns:
        # Vul NULL waarden
        before_nulls = df_cleaned["shortdescription"].isna().sum()
        df_cleaned["shortdescription"] = df_cleaned["shortdescription"].fillna(
            df_cleaned["description"].astype(str).str[:200]
        )
        # Vervang "Ntb"
        ntb_mask = df_cleaned["shortdescription"].str.strip().str.lower() == "ntb"
        df_cleaned.loc[ntb_mask, "shortdescription"] = df_cleaned.loc[ntb_mask, "description"].astype(str).str[:200]
        ntb_replacements["shortdescription"] = ntb_mask.sum()
        print(f"   shortdescription: {before_nulls} NULL gevuld, {ntb_mask.sum()} 'Ntb' vervangen")
    
    # Voor description: gebruik content, of als die ook ntb is, gebruik shortdescription
    if "description" in df_cleaned.columns:
        ntb_mask = df_cleaned["description"].str.strip().str.lower() == "ntb"
        if "content" in df_cleaned.columns:
            # Probeer eerst content
            df_cleaned.loc[ntb_mask, "description"] = df_cleaned.loc[ntb_mask, "content"].astype(str).str[:500]
            # Als content ook 'ntb' is, gebruik shortdescription
            still_ntb = df_cleaned["description"].str.strip().str.lower() == "ntb"
            if "shortdescription" in df_cleaned.columns and still_ntb.sum() > 0:
                df_cleaned.loc[still_ntb, "description"] = df_cleaned.loc[still_ntb, "shortdescription"].astype(str)
        elif "shortdescription" in df_cleaned.columns:
            df_cleaned.loc[ntb_mask, "description"] = df_cleaned.loc[ntb_mask, "shortdescription"].astype(str)
        else:
            df_cleaned.loc[ntb_mask, "description"] = "Geen beschrijving beschikbaar"
        ntb_replacements["description"] = ntb_mask.sum()
        print(f"   description: {ntb_mask.sum()} 'Ntb' vervangen")
    
    # Voor content: gebruik description, of als die ook ntb is, gebruik shortdescription
    if "content" in df_cleaned.columns:
        ntb_mask = df_cleaned["content"].str.strip().str.lower() == "ntb"
        if "description" in df_cleaned.columns:
            # Probeer eerst description
            df_cleaned.loc[ntb_mask, "content"] = df_cleaned.loc[ntb_mask, "description"].astype(str)
            # Als description ook 'ntb' is, gebruik shortdescription
            still_ntb = df_cleaned["content"].str.strip().str.lower() == "ntb"
            if "shortdescription" in df_cleaned.columns and still_ntb.sum() > 0:
                df_cleaned.loc[still_ntb, "content"] = df_cleaned.loc[still_ntb, "shortdescription"].astype(str)
        elif "shortdescription" in df_cleaned.columns:
            df_cleaned.loc[ntb_mask, "content"] = df_cleaned.loc[ntb_mask, "shortdescription"].astype(str)
        else:
            df_cleaned.loc[ntb_mask, "content"] = "Geen content beschikbaar"
        ntb_replacements["content"] = ntb_mask.sum()
        print(f"   content: {ntb_mask.sum()} 'Ntb' vervangen")
    
    # Voor learningoutcomes: gebruik vaste tekst
    if "learningoutcomes" in df_cleaned.columns:
        before_nulls = df_cleaned["learningoutcomes"].isna().sum()
        df_cleaned["learningoutcomes"] = df_cleaned["learningoutcomes"].fillna("Nog niet bepaald")
        ntb_mask = df_cleaned["learningoutcomes"].str.strip().str.lower() == "ntb"
        df_cleaned.loc[ntb_mask, "learningoutcomes"] = "Nog niet bepaald"
        ntb_replacements["learningoutcomes"] = ntb_mask.sum()
        print(f"   learningoutcomes: {before_nulls} NULL gevuld, {ntb_mask.sum()} 'Ntb' vervangen")

    # 3. Vul overige NULL waarden in shortdescription
    if "shortdescription" in df_cleaned.columns and "description" in df_cleaned.columns:
        null_mask = df_cleaned["shortdescription"].isna()
        if null_mask.sum() > 0:
            df_cleaned.loc[null_mask, "shortdescription"] = df_cleaned.loc[null_mask, "description"].astype(str).str[:200]
            print(f"\n3. Resterende NULL waarden in shortdescription aangevuld: {null_mask.sum()}")
    else:
        print("\n3. Geen extra NULL waarden om aan te vullen")

    # 4. start_date naar geldige datetime
    if "start_date" in df_cleaned.columns:
        df_cleaned["start_date"] = pd.to_datetime(df_cleaned["start_date"], errors="coerce")
        invalid_dates = df_cleaned["start_date"].isna().sum()
        print(f"\n4. start_date geconverteerd naar datetime")
        print(f"   Ongeldige datums naar NaT: {invalid_dates}")
    else:
        print("\n4. Kolom start_date ontbreekt, stap overgeslagen")

    # 5. Duplicaten op id droppen
    if "id" in df_cleaned.columns:
        before = df_cleaned.shape[0]
        duplicates = df_cleaned.duplicated(subset=["id"]).sum()
        df_cleaned = df_cleaned.drop_duplicates(subset=["id"])
        after = df_cleaned.shape[0]
        print(f"\n5. Duplicaten op 'id'")
        print(f"   Gevonden: {duplicates}, Rijen voor: {before}, na: {after}")
    else:
        print("\n5. Kolom 'id' ontbreekt, duplicaten-check overgeslagen")

    # 6. Tekstvelden schoonmaken + lemmatizeren
    print("\n6. Tekstvelden normaliseren en lemmatiseren")
    for col in TEXT_COLS:
        if col in df_cleaned.columns:
            clean_col = f"{col}_clean"
            print(f"   - Verwerken: {col} -> {clean_col}")
            df_cleaned[clean_col] = df_cleaned[col].apply(normalize_text)
        else:
            print(f"   - Kolom {col} niet gevonden, overgeslagen")

    # 7. Globale missing value check
    total_nulls = df_cleaned.isnull().sum().sum()
    print("\n" + "=" * 60)
    print("FINALE DATASET STATUS")
    print("=" * 60)
    print(f"Rijen: {df_cleaned.shape[0]}")
    print(f"Kolommen: {df_cleaned.shape[1]}")
    print(f"Totaal NULL waarden: {total_nulls}")

    # Optioneel: per kolom
    print("\nNULL waarden per kolom (alleen > 0):")
    nulls_per_col = df_cleaned.isnull().sum()
    print(nulls_per_col[nulls_per_col > 0])

    # Sample tonen
    sample_cols = [c for c in ["id", "name", "shortdescription", "shortdescription_clean", "learningoutcomes", "learningoutcomes_clean"] if c in df_cleaned.columns]
    print("\nSample (eerste 3 rijen):")
    print(df_cleaned[sample_cols].head(3))

    # Opslaan
    df_cleaned.to_csv(output_file, index=False)
    print(f"\nOPGESLAGEN: {output_file}")

    return df_cleaned


if __name__ == "__main__":
    df_cleaned = clean_vkm_dataset(INPUT_FILE, OUTPUT_FILE)


DATA CLEANING PROCESS
Origineel: 211 rijen, 20 kolommen

1. Kleur-kolommen verwijderd: ['Rood', 'Groen', 'Blauw', 'Geel']
   -> 16 kolommen resterend

2. Vervang 'Ntb' waarden in tekstvelden
   shortdescription: 20 NULL gevuld, 10 'Ntb' vervangen
   description: 2 'Ntb' vervangen
   content: 2 'Ntb' vervangen
   learningoutcomes: 5 NULL gevuld, 26 'Ntb' vervangen

4. start_date geconverteerd naar datetime
   Ongeldige datums naar NaT: 0

5. Duplicaten op 'id'
   Gevonden: 0, Rijen voor: 211, na: 211

6. Tekstvelden normaliseren en lemmatiseren
   - Verwerken: shortdescription -> shortdescription_clean
   - Verwerken: description -> description_clean
   - Verwerken: content -> content_clean
   - Verwerken: learningoutcomes -> learningoutcomes_clean

FINALE DATASET STATUS
Rijen: 211
Kolommen: 20
Totaal NULL waarden: 0

NULL waarden per kolom (alleen > 0):
Series([], dtype: int64)

Sample (eerste 3 rijen):
    id                          name  \
0  159  Kennismaking met Psychologie   
1  

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kloos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kloos\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kloos\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\kloos\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
